In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from datasets import load_dataset
set_seed(3407)

In [2]:
#get the dataset
ds = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", 'plain_text', cache_dir='datasets')

In [6]:
print(ds['train'])

Dataset({
    features: ['text', 'meta'],
    num_rows: 930514
})


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset


# Custom dataset class for the Red Pajama dataset
class RedPajamaDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset['train']  # You can choose 'train', 'test', or 'validation' split here

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        text = sample['text']
        # Convert text and label to tensors if needed
        # For example, if you're using embeddings, you can tokenize the text and convert it to tensor
        # You can also convert labels to tensors if needed
        return text

# Create an instance of the custom dataset
red_pajama_dataset = RedPajamaDataset(ds)

# Create a DataLoader for the custom dataset
batch_size = 32
dataloader = DataLoader(red_pajama_dataset, batch_size=batch_size, shuffle=True)

# # Iterate through the dataloader
# for batch in dataloader:
#     texts = batch['text']
#     labels = batch['label']
#     # Perform your training or evaluation operations using texts and labels tensors


In [23]:
print(red_pajama_dataset[0])

\section{Introduction}
\label{sec:intro}

\emph{Gender diversity}, or more often its lack thereof, among participants to
software development activities has been thoroughly studied in recent years. In
particular, the presence of, effects of, and countermeasures for \emph{gender
  bias} in Free/Open Source Software (FOSS) have received a lot of attention
over the past decade~\cite{david2008fossdevs, qiu2010kdewomen,
  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,
  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,
  zacchiroli2021gender}.  \emph{Geographic diversity} is on the other hand the
kind of diversity that stems from participants in some global activity coming
from different world regions and cultures.

Geographic diversity in FOSS has received relatively little attention in scholarly
works. In particular, while seminal survey-based and
point-in-time medium-scale studies of the geographic origins of FOSS
contributors exist~\cite{ghosh2005understan

In [27]:
model_type = 'gpt2'
prompt = red_pajama_dataset[0]
tokenizer = GPT2Tokenizer.from_pretrained(model_type)
encoded_input = tokenizer(prompt, return_tensors='pt')
x = encoded_input['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (5782 > 1024). Running this sequence through the model will result in indexing errors


In [31]:
print(x.shape)
print(x)

torch.Size([1, 5782])
tensor([[  59, 5458,   90,  ...,  496,  628,  198]])


In [ ]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)